# SOTA Chaser Visualiser
by www.operator-paramedyk.pl / SQ9NIL

explained step-by-step

## What SOTA is?

SOTA (Summits On The Air) is an activity designed for radio amateurs (called also HAMs) who like hiking. It is about communication via ration between an activator - operator who climbed a designated summit (map available at https://sotl.as/map) - and chasers - all other operators. Call  (or QSO) may be done with the use of telegraphy (morse code), voice or data transmission on any band available for radio amateurs. To make a successfull call, both operators need to exchange report, which say how they hear each other, and log it into a log. SOTA results are then uploaded into https://www.sotadata.org.uk/en/ webpage.

You can find all information about SOTA programme at https://www.sota.org.uk/.

This script is designed to visualise on a map all summits chased by an operator. It uses input from log saved in ADIF file (Amateur Data Interchange Format, see specification at https://www.adif.org/303/adif303.htm), gets coordinates of summits chased via SOTA API and plot results on a map.

## Script alghorithm

### Import of relevant packages

There are seven packages to be installed for this script, all specified in requirements.txt file. You can install all of them at once typing in terminal following line:

```pip install -r requirements.txt```

If you use Google Colab (https://colab.research.google.com/) to run this script, uncomment and run the line below to install all packages that are not default in this environment.

In [368]:
# !pip install adif_io
# !pip install unidecode
# !pip install maidenhead

In [369]:
import adif_io as adif # to read the log
import pandas as pd # to analyse log as a DataFrame
import requests # to get data from SOTA API
from unidecode import unidecode # for log clearing
import folium # for data visualisation on a map
import maidenhead as mh # to calculate coordinates from GRID square
import branca.colormap as cm # to add colormap to the visualisation


### Log upload and conversion

You need to save your log as ADIF file. Then copy it into folder where SOTA Visualiser script is saved. As default, script will read SOTAlog.adi file, but you can change this by changing ```filename``` variable value.

There's example log provied with within the repository, which is based on some of my chases for demonstration purposes.

In [370]:
# save name of your log under filename variable
filename = 'SOTAlog.adi'

# working_log will be used to save cleared log
working_log = 'working_log.adi'

Before importing the log, you need to clean it up to avoid errors due to the accents. Therefore, at the beginning, script will ll open the log file and pass it through `unidecode` function to change every local chart to the closest latin one. Then log will be saved under ```working_log.adi``` file.

In [371]:
with open(filename, 'r', encoding='ansi') as filelog:
    content = unidecode(filelog.read())
filelog.close()
with open(working_log, 'w') as filelog:
    filelog.write(content)
filelog.close()    

Now, with the use of adif_id package, your log will be read and divided into two dictionaries:
- ```log_header``` includes ADIF header and it's not used for visualisation, but has to be separated from the QSOs.
- ```log_raw``` includes all your QSOs.

In [372]:
# load log into the variable log_raw, log_header stores header of ADIF file
log_raw,log_header = adif.read_from_file('working_log.adi')

Let's see how both dictionaries look like (for clarity, I present only 1st QSO, remaining ones follows the same structure):

In [373]:
log_header

{'ADIF_VER': '3.1.0', 'PROGRAMID': 'ADIFMaster', 'PROGRAMVERSION': '3.2'}

In [374]:
log_raw[0]

{'STATION_CALLSIGN': 'SQ9NIL',
 'MY_GRIDSQUARE': 'KO00AA',
 'CALL': 'SQ9JTR/P',
 'GRIDSQUARE': 'KN19GC',
 'QTH': 'Wielka Rawka',
 'SOTA_REF': 'SP/BI-003',
 'RST_RCVD': '31',
 'RST_SENT': '53',
 'FREQ': '145.550',
 'BAND': '2M',
 'MODE': 'FM',
 'COMMENT': '145.550',
 'QSO_DATE': '20211106',
 'TIME_ON': '083500',
 'QSO_DATE_OFF': '20211106',
 'TIME_OFF': '083600'}

To make log processing and analysis easier, let's save it as Pandas DataFrame object. First we need to check if there are any chases to visualise - if no, script will cease operation. IF there are SOTA references logged, all of them will be converted to unified format (upper letters only) to avoid duplicate work in coming steps.

In [375]:
# convert the log into DataFrame df_log_raw
df_log_raw = pd.DataFrame(log_raw)

if df_log_raw['SOTA_REF'].count() == 0:
    print('No SOTA chases found in log.')
    exit()
else:
    df_log_raw['SOTA_REF'] = df_log_raw['SOTA_REF'].str.upper()

Now your QSOs are saved in following table (first rows are displayed):

In [376]:
df_log_raw.head()

,STATION_CALLSIGN,MY_GRIDSQUARE,CALL,GRIDSQUARE,QTH,SOTA_REF,RST_RCVD,RST_SENT,FREQ,BAND,MODE,COMMENT,QSO_DATE,TIME_ON,QSO_DATE_OFF,TIME_OFF,MY_SOTA_REF,NAME,OPERATOR,TX_PWR
0,SQ9NIL,KO00AA,SQ9JTR/P,KN19GC,Wielka Rawka,SP/BI-003,31,53,145.550,2M,FM,145.550,20211106,083500,20211106,083600,NaN,NaN,NaN,NaN
1,SQ9NIL,KO00AA,SP9OZI/P,NaN,NaN,SP/BZ-070,59,59,145.550,2M,FM,145.550,20211106,123400,20211106,123500,NaN,NaN,NaN,NaN
2,SQ9NIL,KO00AA,SP9OZI/P,NaN,NaN,SP/BZ-082,57,59,145.550,2M,FM,145.550,20211106,150300,20211106,150400,NaN,NaN,NaN,NaN
3,SQ9NIL,KO00AA,SQ9JTR/P,NaN,NaN,SP/BZ-001,43,59,145.550,2M,FM,145.550,20211111,094100,20211111,094100,NaN,NaN,NaN,NaN
4,SQ9NIL/P,KN09GR,SP9MOV,NaN,NaN,NaN,59,59,145.500,2M,FM,145.500,20211111,111800,20211111,111700,SP/BZ-049,NaN,NaN,NaN


### Prepare data for visualisation

Let's check what is the structure of data in log:

In [377]:
df_log_raw.count()

STATION_CALLSIGN    48
MY_GRIDSQUARE       48
CALL                48
GRIDSQUARE           1
QTH                  3
SOTA_REF            32
RST_RCVD            48
RST_SENT            48
FREQ                48
BAND                48
MODE                48
COMMENT             22
QSO_DATE            48
TIME_ON             48
QSO_DATE_OFF        22
TIME_OFF            48
MY_SOTA_REF         17
NAME                16
OPERATOR            26
TX_PWR              26
dtype: int64

In row count you can see that (in my example) 48 QSOs are saved, but SOTA reference (SOTA_REF row) are provide only for 32 of them. It means that some QSO was not related to SOTA chase. Let's clear the log and remove all records where no SOTA summit reference is provided.

You can check which records include SOTA reference - ```True``` value is returned for them:

In [378]:
df_log_raw.SOTA_REF.notnull()

0      True
1      True
2      True
3      True
4     False
5     False
6     False
7      True
8     False
9     False
10    False
11     True
12    False
13    False
14    False
15    False
16    False
17    False
18    False
19    False
20     True
21     True
22    False
23     True
24     True
25     True
26     True
27     True
28     True
29     True
30     True
31     True
32     True
33     True
34     True
35    False
36     True
37     True
38     True
39     True
40     True
41     True
42     True
43     True
44     True
45     True
46     True
47     True
Name: SOTA_REF, dtype: bool

Now let's remove the rows where SOTA_REF column is empty and name the new DataFrame as df_log_SOTA:

In [379]:
# data filtering - move to further analysis only entries with SOTA reference provided in SOTA_REF column
df_log_SOTA = df_log_raw[df_log_raw.SOTA_REF.notnull()]

New DataFrame includes all columns we had in our log:

In [380]:
df_log_SOTA.head()

,STATION_CALLSIGN,MY_GRIDSQUARE,CALL,GRIDSQUARE,QTH,SOTA_REF,RST_RCVD,RST_SENT,FREQ,BAND,MODE,COMMENT,QSO_DATE,TIME_ON,QSO_DATE_OFF,TIME_OFF,MY_SOTA_REF,NAME,OPERATOR,TX_PWR
0,SQ9NIL,KO00AA,SQ9JTR/P,KN19GC,Wielka Rawka,SP/BI-003,31,53,145.550,2M,FM,145.550,20211106,083500,20211106,083600,NaN,NaN,NaN,NaN
1,SQ9NIL,KO00AA,SP9OZI/P,NaN,NaN,SP/BZ-070,59,59,145.550,2M,FM,145.550,20211106,123400,20211106,123500,NaN,NaN,NaN,NaN
2,SQ9NIL,KO00AA,SP9OZI/P,NaN,NaN,SP/BZ-082,57,59,145.550,2M,FM,145.550,20211106,150300,20211106,150400,NaN,NaN,NaN,NaN
3,SQ9NIL,KO00AA,SQ9JTR/P,NaN,NaN,SP/BZ-001,43,59,145.550,2M,FM,145.550,20211111,094100,20211111,094100,NaN,NaN,NaN,NaN
7,SQ9NIL/P,KN09GR,SQ9PPW/P,NaN,NaN,SP/BZ-059,59,59,145.500,2M,FM,"145.500, S2S",20211111,112400,20211111,112500,SP/BZ-049,NaN,NaN,NaN


But all records now have SOTA summit reference included:

In [381]:
df_log_SOTA.count()

STATION_CALLSIGN    32
MY_GRIDSQUARE       32
CALL                32
GRIDSQUARE           1
QTH                  1
SOTA_REF            32
RST_RCVD            32
RST_SENT            32
FREQ                32
BAND                32
MODE                32
COMMENT              8
QSO_DATE            32
TIME_ON             32
QSO_DATE_OFF         8
TIME_OFF            32
MY_SOTA_REF          3
NAME                12
OPERATOR            24
TX_PWR              24
dtype: int64

### Get SOTA summits data from SOTA API

Now, we will use SOTA API (https://api2.sota.org.uk/docs/index.html) to get data of summits chased.

We'll check which specific summits are present in the log uploaded. To do this, we create a new Series containing data from SOTA_REF column of the log with duplicates removed.

In [382]:
df_log_summits = df_log_SOTA['SOTA_REF'].copy().drop_duplicates().reset_index()
del(df_log_summits['index'])
df_log_summits


,SOTA_REF
0,SP/BI-003
1,SP/BZ-070
2,SP/BZ-082
3,SP/BZ-001
4,SP/BZ-059
5,SP/WS-003
6,OM/PO-040
7,SP/BZ-010
8,SP/BZ-030
9,SP/BZ-024


With the list of summits in scope, we can go to the SOTA API and download data for every summit chased into ```summits_dict``` dictionary. If there are any exceptions (e.g. due to the incorrectly logged summit reference), summit reference with error code will be saved in ```errors_dict``` dictionary and displayed to notify user about the errors found. 

In [383]:
summits_dict={}
errors_dict = {}
url = 'https://api2.sota.org.uk/api/summits/'

for summit in df_log_summits['SOTA_REF']:
    summit_url = f"{url}{summit}"
    try:
        r = requests.get(summit_url)
        print(f'Status code: {r.status_code} for {summit}')
        summits_dict[summit] = r.json()
    except Exception:
        errors_dict[summit] = r.status_code
        pass

if errors_dict:
    print(f'\nFollowing errors were reported - QSO will be not included')
    for summit in errors_dict.keys():
        print(f'Error {errors_dict[summit]} for {summit}.')
        df_log_summits = df_log_summits.loc[df_log_summits['SOTA_REF'] != summit]
        
        
    

Status code: 200 for SP/BI-003
Status code: 200 for SP/BZ-070
Status code: 200 for SP/BZ-082
Status code: 200 for SP/BZ-001
Status code: 200 for SP/BZ-059
Status code: 200 for SP/WS-003
Status code: 200 for OM/PO-040
Status code: 200 for SP/BZ-010
Status code: 200 for SP/BZ-030
Status code: 200 for SP/BZ-024
Status code: 200 for SP/BZ-031
Status code: 200 for SP/BZ-005
Status code: 200 for SP/BZ-014


Similarly to the log, let's save the summits data downloaded as DataFrame object.

In [384]:
df_summits = pd.DataFrame(summits_dict)
df_summits

,SP/BI-003,SP/BZ-070,SP/BZ-082,SP/BZ-001,SP/BZ-059,SP/WS-003,OM/PO-040,SP/BZ-010,SP/BZ-030,SP/BZ-024,SP/BZ-031,SP/BZ-005,SP/BZ-014
summitCode,SP/BI-003,SP/BZ-070,SP/BZ-082,SP/BZ-001,SP/BZ-059,SP/WS-003,OM/PO-040,SP/BZ-010,SP/BZ-030,SP/BZ-024,SP/BZ-031,SP/BZ-005,SP/BZ-014
name,Wielka Rawka,Kostrza,Zęzów,Diablak (Babia Góra),Kotoń,Góra Zamkowa (Góra Janowskiego),Javorina,Gorc (Gorc Kamieniecki),Modyń,Ćwilin,Luboń (Luboń Wielki),Turbacz,Mogielica
shortCode,BI-003,BZ-070,BZ-082,BZ-001,BZ-059,WS-003,PO-040,BZ-010,BZ-030,BZ-024,BZ-031,BZ-005,BZ-014
altM,1307,730,693,1725,857,516,881,1228,1028,1072,1022,1315,1171
altFt,4288,2395,2273,5659,2812,1691,2890,4029,3373,3517,3353,4314,3842
gridRef1,22.5780,20.2993,20.3198,19.5296,19.8961,19.5536,21.2637,20.2528,20.376,20.1916,19.9919,20.1113,20.2768
gridRef2,49.0988,49.7708,49.7467,49.5732,49.7689,50.4511,49.4473,49.5653,49.6212,49.6887,49.6535,49.5429,49.6552
notes,,,,,,,,,,,,,
validFrom,0001-01-01T00:00:00,0001-01-01T00:00:00,0001-01-01T00:00:00,0001-01-01T00:00:00,0001-01-01T00:00:00,0001-01-01T00:00:00,0001-01-01T00:00:00,0001-01-01T00:00:00,0001-01-01T00:00:00,0001-01-01T00:00:00,0001-01-01T00:00:00,0001-01-01T00:00:00,0001-01-01T00:00:00
validTo,0001-01-01T00:00:00,0001-01-01T00:00:00,0001-01-01T00:00:00,0001-01-01T00:00:00,0001-01-01T00:00:00,0001-01-01T00:00:00,0001-01-01T00:00:00,0001-01-01T00:00:00,0001-01-01T00:00:00,0001-01-01T00:00:00,0001-01-01T00:00:00,0001-01-01T00:00:00,0001-01-01T00:00:00


As you can see, dataframe `df_summits` includes field `myChases`, currently filled with `None` values. We can use this field to save number of chases for each summit.

In [385]:
for summit in df_log_summits['SOTA_REF']:
    df_summits[summit]['myChases'] = df_log_SOTA[['SOTA_REF']][df_log_SOTA['SOTA_REF'] == summit].count()[0]

In [386]:
df_summits.keys

<bound method NDFrame.keys of                            SP/BI-003            SP/BZ-070  \
summitCode                 SP/BI-003            SP/BZ-070   
name                    Wielka Rawka              Kostrza   
shortCode                     BI-003               BZ-070   
altM                            1307                  730   
altFt                           4288                 2395   
gridRef1                     22.5780              20.2993   
gridRef2                     49.0988              49.7708   
notes                                                       
validFrom        0001-01-01T00:00:00  0001-01-01T00:00:00   
validTo          0001-01-01T00:00:00  0001-01-01T00:00:00   
activationCount                 None                 None   
myActivations                   None                 None   
activationDate                  None                 None   
myChases                           1                    1   
activationCall                  None                 No

Before visualisation, we'll do one more step to make - transpond the ```df_summits```  DataFrame so each column (Series object) consist of the same type of data.

In [387]:
df_summits_transposed = df_summits.transpose(copy = True)
df_summits_transposed

,summitCode,name,shortCode,altM,altFt,gridRef1,gridRef2,notes,validFrom,validTo,...,latitude,locator,points,regionCode,regionName,associationCode,associationName,valid,restrictionMask,restrictionList
SP/BI-003,SP/BI-003,Wielka Rawka,BI-003,1307,4288,22.5780,49.0988,,0001-01-01T00:00:00,0001-01-01T00:00:00,...,49.0988,KN19gc,10,BI,Bieszczady,SP,Poland,True,False,[]
SP/BZ-070,SP/BZ-070,Kostrza,BZ-070,730,2395,20.2993,49.7708,,0001-01-01T00:00:00,0001-01-01T00:00:00,...,49.7708,KN09ds,4,BZ,Beskidy Zachodnie,SP,Poland,True,False,[]
SP/BZ-082,SP/BZ-082,Zęzów,BZ-082,693,2273,20.3198,49.7467,,0001-01-01T00:00:00,0001-01-01T00:00:00,...,49.7467,KN09dr,2,BZ,Beskidy Zachodnie,SP,Poland,True,False,[]
SP/BZ-001,SP/BZ-001,Diablak (Babia Góra),BZ-001,1725,5659,19.5296,49.5732,,0001-01-01T00:00:00,0001-01-01T00:00:00,...,49.5732,JN99sn,10,BZ,Beskidy Zachodnie,SP,Poland,True,False,[]
SP/BZ-059,SP/BZ-059,Kotoń,BZ-059,857,2812,19.8961,49.7689,,0001-01-01T00:00:00,0001-01-01T00:00:00,...,49.7689,JN99ws,6,BZ,Beskidy Zachodnie,SP,Poland,True,False,[]
SP/WS-003,SP/WS-003,Góra Zamkowa (Góra Janowskiego),WS-003,516,1691,19.5536,50.4511,,0001-01-01T00:00:00,0001-01-01T00:00:00,...,50.4511,JO90sk,1,WS,Wyzyna Slaska,SP,Poland,True,False,[]
OM/PO-040,OM/PO-040,Javorina,PO-040,881,2890,21.2637,49.4473,,0001-01-01T00:00:00,0001-01-01T00:00:00,...,49.4473,KN09pk,2,PO,Prešovský,OM,Slovakia,True,False,[]
SP/BZ-010,SP/BZ-010,Gorc (Gorc Kamieniecki),BZ-010,1228,4029,20.2528,49.5653,,0001-01-01T00:00:00,0001-01-01T00:00:00,...,49.5653,KN09dn,8,BZ,Beskidy Zachodnie,SP,Poland,True,False,[]
SP/BZ-030,SP/BZ-030,Modyń,BZ-030,1028,3373,20.376,49.6212,,0001-01-01T00:00:00,0001-01-01T00:00:00,...,49.6212,KN09eo,8,BZ,Beskidy Zachodnie,SP,Poland,True,False,[]
SP/BZ-024,SP/BZ-024,Ćwilin,BZ-024,1072,3517,20.1916,49.6887,,0001-01-01T00:00:00,0001-01-01T00:00:00,...,49.6887,KN09cq,8,BZ,Beskidy Zachodnie,SP,Poland,True,False,[]


DataFrame now looks good for visualisation however we cannot use the data as they are not numeric:

In [388]:
df_summits_transposed.dtypes

summitCode         object
name               object
shortCode          object
altM               object
altFt              object
gridRef1           object
gridRef2           object
notes              object
validFrom          object
validTo            object
activationCount    object
myActivations      object
activationDate     object
myChases           object
activationCall     object
longitude          object
latitude           object
locator            object
points             object
regionCode         object
regionName         object
associationCode    object
associationName    object
valid              object
restrictionMask    object
restrictionList    object
dtype: object

Actually we need only four numbers for visualisation:
- summit's reference: latitude and longitude,
- numbers of chases for each summit,
- summit's SOTA points value,

so we can convert respective values:

In [389]:
df_summits_transposed['latitude'] = df_summits_transposed['latitude'].astype('float')
df_summits_transposed['longitude'] = df_summits_transposed['longitude'].astype('float')
df_summits_transposed['myChases'] = df_summits_transposed['myChases'].astype('int')
df_summits_transposed['points'] = df_summits_transposed['points'].astype('int')
df_summits_transposed.dtypes

summitCode          object
name                object
shortCode           object
altM                object
altFt               object
gridRef1            object
gridRef2            object
notes               object
validFrom           object
validTo             object
activationCount     object
myActivations       object
activationDate      object
myChases             int32
activationCall      object
longitude          float64
latitude           float64
locator             object
points               int32
regionCode          object
regionName          object
associationCode     object
associationName     object
valid               object
restrictionMask     object
restrictionList     object
dtype: object

Now we have everything we need regarding summits chased, but I think it's also good to include chaser's location on a map. Amateur radio operators use QTH locator grid (https://en.wikipedia.org/wiki/Maidenhead_Locator_System) and this data is available in ```MY_GRIDSQUARE``` field in the log.

Grid locator in a form of two letters - two digits - two letters provides information about station's location with accuracy c.a. 5 km and may be calculated for geographical latitude and longituted. Here I'll use ```to_location``` function from ```maindenhead``` package to calculate coordinates and save it in the list.

For every HAM there's one very personal place - home QTH. Based on the log entries, we can guess that it will be the most common grid locator (I know it's not always true, especially for operators usually operating from field). It will serves as a map center for us. If there is no chaser's QTH logged (which is possible), map will be centred on the most frequently chased summit.

In [390]:
## prepare list of chasing locations if any
my_coordinates = []

if ('MY_GRIDSQUARE' in df_log_SOTA) == True:

    for locator in df_log_SOTA['MY_GRIDSQUARE'].drop_duplicates():
            my_coordinates.append(list(mh.to_location(locator)))
    home_QTH = list(mh.to_location(df_log_SOTA['MY_GRIDSQUARE'].value_counts().idxmax()))
    map_center = home_QTH
else:
    map_center = [
        df_summits_transposed['latitude'][df_summits_transposed['myChases'].idxmax()],
        df_summits_transposed['longitude'][df_summits_transposed['myChases'].idxmax()]
    ]

 To make visualisation easier-to-read, I'll also standardise the count of QSO for each summit (so the value will be equal to 1 for most chased summit, and proportionally lower to remaining ones) and save it in newly created ```rel_Chases``` column.

In [391]:
df_summits_transposed['rel_Chases'] = df_summits_transposed['myChases']/df_summits_transposed['myChases'].max()
df_summits_transposed['rel_Chases']

SP/BI-003    0.0625
SP/BZ-070    0.0625
SP/BZ-082    0.0625
SP/BZ-001    0.0625
SP/BZ-059    0.0625
SP/WS-003    0.0625
OM/PO-040    0.0625
SP/BZ-010    0.0625
SP/BZ-030    1.0000
SP/BZ-024    0.1250
SP/BZ-031    0.1250
SP/BZ-005    0.1250
SP/BZ-014    0.1250
Name: rel_Chases, dtype: float64

### Data visualisation

Now we can visualise chases avaiable in the log on a map with a support of Folium library. Before we start, I'll define a colormap to reflect summit's punctation. Based on a region and height, each summit is valued between 1 and 10 points. We'll use colormap where 1-points summits are marked with purple, 5-points ones - with orange, and 10-points ones - with red. 

In [392]:
summit_points = cm.LinearColormap(colors=['purple', 'orange','red'], index=[1,5,10],vmin=1,vmax=10).to_step(10)

Let's draw a map centred on home QTH.

In [393]:
# prepare a map 
chasers_map = folium.Map(location=map_center, 
                         tiles="Stamen Terrain",
                        zoom_start=9)

Every summit chased is presented on a map with a circle centered at summit's coordinates. Circle's color represents it's points value, and diameter - how many chases were made with this particular summit.

In [394]:
# add summits to a map
for summit in df_log_summits['SOTA_REF']:
    folium.CircleMarker(
    location = [df_summits_transposed['latitude'][summit], df_summits_transposed['longitude'][summit]],
    radius = 20*df_summits_transposed['rel_Chases'][summit],
    popup = f"{df_summits_transposed['summitCode'][summit]},\n{df_summits_transposed['name'][summit]}\n{df_summits_transposed['myChases'][summit]} QSOs",
    color = summit_points(df_summits_transposed['points'][summit]),
    fill = True,
    weight = 0,
    fill_opacity = 1
).add_to(chasers_map)

Home QTH is presented with home marker.

In [395]:
# add home marker to map
if ('MY_GRIDSQUARE' in df_log_SOTA) == True:
    folium.Marker(
        location=home_QTH,
        popup=f"home QTH: {df_log_SOTA['MY_GRIDSQUARE'].value_counts().idxmax()}",
        icon=folium.Icon(color="blue", icon="glyphicon-home"),
    ).add_to(chasers_map)

... and every other locator - with a blue dot.

In [396]:
# add your locations to map
for coordinate in my_coordinates:
    if coordinate == map_center:
        pass
    else:
        folium.CircleMarker(
            location = coordinate,
            popup = 'field QTH',
            color = 'dodgerblue',
            fill = True,
            fill_opacity = 1,
            weight = 0,
            radius = 5
        ).add_to(chasers_map)

Finally, we can add colormap bar into a map and print it.

In [397]:
chasers_map.add_child(summit_points)
# print map
display(chasers_map)